# Clasificar proyectos ANID según tipo de enfermedad

El objetivo de este notebook es clasificar cada proyecto ANID de acuerdo a la enfermedad abordada en el proyecto: `cáncer`, `diabetes`, `cardiaca` (para enfermedades cardiovasculares) u `otro`, si es que no trata ninguna de las enfermedades señaladas.

In [ ]:
!pip install openai
!pip install langchain
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import numpy as np
import pandas as pd
from csv import writer
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata

## Cargar datos

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Datos"
save_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Resultados"
df = pd.read_excel(os.path.join(data_dir, "BDH_HISTORICA.xlsx"))
df

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NO APLICA,36.0,PERSONA NATURAL,JIMENA TORRES,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,9280,NaN
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NO APLICA,24.0,PERSONA JURIDICA,FABIANA MARTIN,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,12005.582,NaN
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,LAURA SANCHEZ-JARDON,MUJER,FUNDACION OMORA ONG,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,SERGIO RADIC,HOMBRE,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NO APLICA,12.0,PERSONA NATURAL,PAULINA ROJAS,MUJER,INSTITUTO ANTARTICO CHILENO (INACH),AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,22000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42073,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"PIGNATA, GIULIANO",HOMBRE,UNIVERSIDAD NACIONAL ANDRÉS BELLO,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42074,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"RAJKUMAR, ANITHA RAJ",HOMBRE,UNIVERSIDAD DE ATACAMA,NORTE,03. ATACAMA,0,NO ENTREGA RECURSOS
42075,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"NAYAK, PRASANTA KUMAR",MUJER,PONTIFICIA UNIVERSIDAD CATÓLICA DE CHILE,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42076,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"TEJOS, NICOLAS",HOMBRE,PONTIFICIA UNIVERSIDAD CATÓLICA DE VALPARAÍSO,CENTRO,05. VALPARAISO,0,NO ENTREGA RECURSOS


## Preprocesar datos

In [ ]:
df.columns

Index(['CODIGO_PROYECTO', 'N', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO'],
      dtype='object')

In [ ]:
df[df["NOMBRE_PROYECTO"].isna()]

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
10595,BDE041,10564,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,MULTIDISCIPLINARIO,...,NO APLICA,NaN,PERSONA NATURAL,ANGELA MARCELA GANZ BUSTOS,MUJER,INRIA SOPHIA-ANTIPOLIS,EXTRANJERO,EXTRANJERO,NaN,NaN
10596,BDE0410,10565,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,CLAUDIA ANDREA MARQUEZ THOMAS,MUJER,UNIVERSIDAD PARIS 7 DENIS DIDEROT,EXTRANJERO,EXTRANJERO,NaN,NaN
10597,BDE0411,10566,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,CIENCIAS AGRICOLAS,...,NO APLICA,NaN,PERSONA NATURAL,MARITZA REGINA REYES CARREÑO,MUJER,INSTITUTO NACIONAL DE INVESTIGACION AGRONOMICA...,SIN INFORMACION,SIN INFORMACION,NaN,NaN
10598,BDE0412,10567,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,LUIS CRISTOBAL ROJAS GONZALEZ,HOMBRE,UNIVERSIDAD PIERRE ET MARIE CURIE,EXTRANJERO,EXTRANJERO,NaN,NaN
10599,BDE0413,10568,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADO EN EL EXTRANJERO,CONCURSO DE BECAS DE DOCTORADO EN EL EXTRANJER...,2004,2004,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,ESTEBAN PATRICIO SAEZ ROBERT,HOMBRE,ESCUELA CENTRAL PARIS,EXTRANJERO,EXTRANJERO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12977,BDN0695,12946,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS AGRICOLAS,...,NO APLICA,NaN,PERSONA NATURAL,CLAUDIA ISABELHARCHA CORTES,MUJER,UNIVERSIDAD AUSTRAL DE CHILE,SUR,14. LOS RIOS,NaN,NaN
12978,BDN0696,12947,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,DENISE ALICIA HAUSSMANN BIELEFELD,MUJER,UNIVERSIDAD AUSTRAL DE CHILE,SUR,14. LOS RIOS,NaN,NaN
12979,BDN0697,12948,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,CIENCIAS NATURALES,...,NO APLICA,NaN,PERSONA NATURAL,RICARDO ANDRES HENRIQUEZ CORREA,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,NaN,NaN
12980,BDN0698,12949,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,BECAS DE DOCTORADOS NACIONALES,CONCURSO DE BECAS DE DOCTORADOS NACIONALES AÑO...,2006,2006,NaN,INGENIERIA Y TECNOLOGIA,...,NO APLICA,NaN,PERSONA NATURAL,LUIS ANDRES HENRIQUEZ VARGAS,HOMBRE,UNIVERSIDAD DE SANTIAGO DE CHILE,RM,13. METROPOLITANA,NaN,NaN


Tenemos 574 filas sin nombre de proyecto; parecen ser de adjudicaciones de becas. Además, a diferencia de CORFO, no contamos con la descripción de los proyectos, sino que sólo tenemos los títulos.

In [ ]:
df_valid = df[~df["NOMBRE_PROYECTO"].isna()]
len(df_valid)

41504

## Clasificación

Inicializar modelo

In [ ]:
api_key = userdata.get('openai_api_key')
chat = ChatOpenAI(model_name="gpt-4",
                    openai_api_key=api_key,
                    temperature=0,
                    max_tokens=3500)
chat

Le daremos al LLM una lista de instrucciones, pidiéndole clasificar cada proyecto en `cáncer`, `diabetes`, `cardiaca` u `otro`. Especificaremos que los proyectos pueden estar en inglés o español. Luego, le pasaremos una lista de proyectos para que clasifique. Iteraremos en bloques de 10 proyectos, e iremos guardando de inmediato los resultados.

In [ ]:
system_prompt = ("Eres un experto en evaluación de proyectos de investigación. " +
                 "Recibiras una lista de proyectos, cada uno empezando con un guión (-). " +
                 "Los proyectos sólo tienen título, y estos pueden estar en inglés o español. " +
                 "Tu trabajo es determinar para cada proyecto si habla de cáncer, de diabetes, de enfermedades cardiacas u otro tema. " +
                 "Si el tema del proyecto es cáncer, debes responder 'cáncer'. " +
                 "Si el tema del proyecto es diabetes, debes responder 'diabetes'. " +
                 "Si el tema del proyecto es enfermedades cardiacas, debes responder 'cardiaca'. " +
                 "Si el tema del proyecto es algo distinto, debes responder 'otro.'. " +
                 "No pongas guiones al principio de cada respuesta. Sólo debes responder la clasificación correcta en cada línea.")

system_message_prompt = SystemMessage(content=system_prompt)
n_projects_prompt = 10

filepath = os.path.join(save_dir, "clasificaciones_anid_enfermedades_especificas.csv")
if not os.path.exists(filepath):
  with open(filepath, "w") as fp:
    print("Creando archivo...")
    csv_writer = writer(fp)
    header = ["código", "glosa", "clasificación"]
    csv_writer.writerow(header)


for iii in tqdm(range(0, len(df_valid), n_projects_prompt)):
  current_df = df_valid.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii+1)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

Creando archivo...


  7%|▋         | 276/4151 [15:28<3:34:19,  3.32s/it]

Distinto número de respuestas que de proyectos
2750 2751


 25%|██▍       | 1035/4151 [54:49<2:20:20,  2.70s/it]

Distinto número de respuestas que de proyectos
10340 10341


...

 83%|████████▎ | 3461/4151 [3:01:48<33:36,  2.92s/it]

Distinto número de respuestas que de proyectos
34600 34601


 91%|█████████ | 3764/4151 [3:17:19<20:37,  3.20s/it]

Distinto número de respuestas que de proyectos
37630 37631


100%|██████████| 4151/4151 [3:37:00<00:00,  3.14s/it]


### Datos faltantes

Vemos que hubo algunas iteraciones que fallaron, así que intentaremos de nuevo procesar esos datos.

In [ ]:
df_missing = df_valid.iloc[np.r_[2750:2760, 10340:10350, 31330:31340, 34600:34610, 37630:37640]]
df_missing

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
2750,D91I1040,2719,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,SIN INFORMACION,INGENIERIA Y TECNOLOGIA,...,NO APLICA,36.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN
2751,D91I1042,2720,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,VALORIZACION DEL BOSQUE NATIVO:TIPOS FORESTALE...,INGENIERIA Y TECNOLOGIA,...,NO APLICA,43.0,PERSONA JURIDICA,MARIO HERMOSILLA DAUDET,HOMBRE,FUNDACION CHILE,RM,13. METROPOLITANA,387000,NaN
2752,D91I1043,2721,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,SIN INFORMACION,INGENIERIA Y TECNOLOGIA,...,NO APLICA,48.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN
2753,D91I1045,2722,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,DESARROLLO DE SISTEMAS Y MODELOS DE OPTIMIZACI...,CIENCIAS NATURALES,...,NO APLICA,49.0,PERSONA JURIDICA,ANDRES WEINTRAUB POHORILLE,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,293000,NaN
2754,D91I1046,2723,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,SIN INFORMACION,CIENCIAS AGRICOLAS,...,NO APLICA,48.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN
2755,D91I1047,2724,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,LA MEJORA GENETICA DE LOS EUCALIPTOS EN CHILE,CIENCIAS AGRICOLAS,...,NO APLICA,51.0,PERSONA JURIDICA,NORMAN SMITH VIVALLO,HOMBRE,INSTITUTO FORESTAL (INFOR),RM,13. METROPOLITANA,486000,NaN
2756,D91I1048,2725,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,SIN INFORMACION,CIENCIAS AGRICOLAS,...,NO APLICA,48.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN
2757,D91I1049,2726,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,BIOBLANQUEAMIENTO DE PULPA DE CELULOSA KRAFT Y...,CIENCIAS NATURALES,...,NO APLICA,36.0,PERSONA JURIDICA,JOSE VICUÑA ERRAZURIZ,HOMBRE,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,RM,13. METROPOLITANA,288000,NaN
2758,D91I1050,2727,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,SIN INFORMACION,CIENCIAS NATURALES,...,NO APLICA,48.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN
2759,D91I1051,2728,INVESTIGACION APLICADA,FONDEF,INVESTIGACION Y DESARROLLO,I CONCURSO INVESTIGACION Y DESARROLLO (I+D) 1991,1991,1991,TEC SILVICOLAS Y PRODUCTIVIDAD DEL BOSQUE NATC...,CIENCIAS AGRICOLAS,...,NO APLICA,48.0,PERSONA JURIDICA,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN


Una hipótesis inicial es que el proceso falló en esas iteraciones porque el prompt era muy largo. Así que, en vez de grupos de 10 proyectos, iteraremos en grupos de 5.

In [ ]:
n_projects_prompt = 5
system_message_prompt = SystemMessage(content=system_prompt)

for iii in tqdm(range(0, len(df_missing), n_projects_prompt)):
  current_df = df_missing.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + n_projects_prompt)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

 30%|███       | 3/10 [00:05<00:13,  1.91s/it]

Distinto número de respuestas que de proyectos
10 20


 50%|█████     | 5/10 [00:12<00:13,  2.64s/it]

Distinto número de respuestas que de proyectos
20 30


100%|██████████| 10/10 [00:24<00:00,  2.45s/it]


Dos iteraciones fallaron

In [ ]:
df_missing2 = df_missing.iloc[np.r_[10:20, 20:30]]
df_missing21010

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
10340,CET08,10309,NO APLICA,PROGRAMA BICENTENARIO DE CIENCIA Y TECNOLOGIA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO,I CONCURSO DE CAMPAMENTOS DE EMPRENDIMIENTO TE...,2004,2004,S/I,CIENCIAS AGRICOLAS,...,NO APLICA,NaN,PERSONA JURIDICA,FELIPE GALLARDO ARRIAGADA,HOMBRE,UNIVERSIDAD DE LA FRONTERA,SUR,09. LA ARAUCANIA,7000,NaN
10341,ECT1/04/02,10310,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,TERRAZAS ANDINAS: UNA PUERTA AL PASADO CON VIS...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,MERCEDES ELIANA BELMONTE SCHWARZBAUM,MUJER,UNIVERSIDAD DE TARAPACA,NORTE,01. TARAPACA,7000,NaN
10342,ECT1/04/03,10311,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,VENTANA INTERACTIVA AL UNIVERSO,CIENCIAS NATURALES,...,NO APLICA,11.0,PERSONA JURIDICA,JAVIER RUIZ DEL SOLAR SAN MARTIN,HOMBRE,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,7000,NaN
10343,ECT1/04/04,10312,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,USANDO LA CREATIVIDAD LACIENCIA Y LA TECNOLOGI...,CIENCIAS NATURALES,...,NO APLICA,11.0,PERSONA JURIDICA,MARGARITA MALVINA GARRIDO ESPINOZA,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,6734,NaN
10344,ECT1/04/05,10313,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,CAMPAMENTOS TECNOLOGICOS KAWIN MELI KUREF EN E...,CIENCIAS NATURALES,...,NO APLICA,11.0,PERSONA JURIDICA,FELIPE OCTAVIO GALLARDO ARRIAGADA,HOMBRE,UNIVERSIDAD DE LA FRONTERA,SUR,09. LA ARAUCANIA,7000,NaN
10345,ECT1/04/06,10314,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,CAMPAMENTO TECNOLOGICO DE ELECTROMAQUINAS,CIENCIAS SOCIALES,...,NO APLICA,11.0,PERSONA JURIDICA,MARCELO ANDRES MUNZENMAYER SCHULLER,HOMBRE,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,RM,13. METROPOLITANA,4690,NaN
10346,ECT1/04/07,10315,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,SOLUCIONES TECNOLOGICAS PARA EL DESARROLLO DE ...,CIENCIAS NATURALES,...,NO APLICA,11.0,PERSONA JURIDICA,ANITA MATILDE VALDES JAHNSEN,MUJER,UNIVERSIDAD DE CONCEPCION,CENTRO SUR,08. BIOBIO,6975,NaN
10347,ECT1/04/08,10316,NO APLICA,EXPLORA,CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PARA...,I CAMPAMENTOS DE EMPRENDIMIENTO TECNOLOGICO PA...,2004,2004,ELABORACION DE UNA ENCICLOPEDIA MULTIMEDIA DE ...,CIENCIAS NATURALES,...,NO APLICA,11.0,PERSONA JURIDICA,GUILLERMO EDUARDO LOPEZ QUINTANA,HOMBRE,UNIVERSIDAD CATOLICA DEL MAULE,CENTRO SUR,07. MAULE,5990,NaN
10348,SA04I2005,10317,INVESTIGACION APLICADA,FONIS,CONCURSO NACIONAL DE PROYECTOS DE INVESTIGACIO...,I CONCURSO NACIONAL DE PROYECTOS DE INVESTIGAC...,2004,2004,ATLAS DE MORTALIDAD POR ENFERMEDADES CARDIOVAS...,CIENCIAS MEDICAS Y DE LA SALUD,...,NO APLICA,15.0,PERSONA JURIDICA,MARIA GLORIA ICAZA NOGUERA,MUJER,UNIVERSIDAD DE TALCA,CENTRO SUR,07. MAULE,9000,NaN
10349,SA04I2009,10318,INVESTIGACION APLICADA,FONIS,CONCURSO NACIONAL DE PROYECTOS DE INVESTIGACIO...,I CONCURSO NACIONAL DE PROYECTOS DE INVESTIGAC...,2004,2004,INVESTIGACION BUENAS PRACTICAS DE PARTICIPACIO...,CIENCIAS MEDICAS Y DE LA SALUD,...,NO APLICA,15.0,PERSONA JURIDICA,ALEJANDRO AGUSTIN MONTERO CORNEJO,HOMBRE,ORGANIZACION NO GUBERNAMENTAL CENTRO DE EDUCAC...,SUR,09. LA ARAUCANIA,16000,NaN


In [ ]:
df_missing2["NOMBRE_PROYECTO"].str.len()

10340      3
10341     59
10342     31
10343     85
10344     95
10345     41
10346     91
10347    144
10348     70
10349    118
31904     81
31905     88
31906     45
31907     27
31908     97
31909     74
31910     42
31911     70
31912     47
31913     61
Name: NOMBRE_PROYECTO, dtype: int64

El proyecto 10340 tiene un título de 3 caracteres: "S/I", así que lo descartaremos.


In [ ]:
df_missing2 = df_missing2.drop(10340)

In [ ]:
len(df_missing2)

19

Probaremos de nuevo

In [ ]:
for iii in tqdm(range(0, len(df_missing2), n_projects_prompt)):
  current_df = df_missing2.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

 75%|███████▌  | 3/4 [00:09<00:03,  3.15s/it]

Distinto número de respuestas que de proyectos
10 20


100%|██████████| 4/4 [00:10<00:00,  2.60s/it]


Ahora hay sólo una iteración con problemas.

In [ ]:
df_missing3 = df_missing2.iloc[10:15]
n_projects_prompt = 1

for iii in tqdm(range(0, len(df_missing3), n_projects_prompt)):
  current_df = df_missing3.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["NOMBRE_PROYECTO"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["CODIGO_PROYECTO"].values, current_df["NOMBRE_PROYECTO"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

100%|██████████| 5/5 [00:04<00:00,  1.13it/s]


## Revisar clasificaciones

In [ ]:
df_class = pd.read_csv(filepath)
df_class

,código,glosa,clasificación
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro
4,ED230064,PODCAST ANTARTICA,otro
...,...,...,...
41510,31180032,DEVELOPING ASTRONOMY TOOLS FOR BLIND AND VISUA...,otro.
41511,31180035,STAR FORMATION THROUGH TIME: FROM NEAR TO FAR,otro.
41512,31180046,ANNUAL MEETING SOCHIAS 2020,otro.
41513,31180047,"""ASTROSERENA - EXPLORANDO EL COSMOS DESDE LA C...",otro.


In [ ]:
df_class["clasificación"].value_counts()

otro          32663
otro.          6393
- otro          839
cáncer          767
cardiaca        399
diabetes        269
- cáncer         53
- cardiaca       42
cardiaca.        35
cáncer.          33
- diabetes       16
diabetes.         6
Name: clasificación, dtype: int64

In [ ]:
df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()
df_class["clasificación_normalizada"].value_counts()

<ipython-input-41-b6ce80997fd7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()


otro        39895
cáncer        853
cardiaca      476
diabetes      291
Name: clasificación_normalizada, dtype: int64

### Revisar duplicados

In [ ]:
df_class[df_class.duplicated()]

,código,glosa,clasificación,clasificación_normalizada
11581,SIN INFORMACION,SIN INFORMACION,otro,otro
12278,SIN INFORMACION,CHUNGARA,otro,otro
12283,SIN INFORMACION,REVISTA SIGNOS,otro,otro
12284,SIN INFORMACION,ELECTRONIC JOURNAL OF BIOTECHNOLOGY,otro,otro
12286,SIN INFORMACION,ULTIMA DECADA,otro,otro
...,...,...,...,...
41502,SA04I2009,INVESTIGACION BUENAS PRACTICAS DE PARTICIPACIO...,otro,otro
41504,31180050,REVEALING THE DYNAMICS OF PLANET FORMATION,otro,otro
41505,31180051,STRUCTURE AND DYNAMICS OF GALAXIES ACROSS THE ...,otro,otro
41506,31180052,STUDYING THE DAWN OF PLANETS IN THE ERA OF ALMA,otro,otro


Notamos que hay varios códigos "SIN INFORMACIÓN" que no habíamos notado. Revisemos eso primero.

In [ ]:
df_valid["CODIGO_PROYECTO"].value_counts()

SIN INFORMACION    173
VIU15P0019           2
VIU15P0036           2
VIU15P0047           2
VIU17P0067           2
                  ... 
2007 – 194           1
2007-142             1
2007-152             1
2007-169             1
CL-2023B-018         1
Name: CODIGO_PROYECTO, Length: 41240, dtype: int64

In [ ]:
df_valid.loc[df_valid["CODIGO_PROYECTO"] == "SIN INFORMACION", ["NOMBRE_PROYECTO", "AGNO_FALLO"]]

,NOMBRE_PROYECTO,AGNO_FALLO
11443,ACTA LITERARIA,2005
11444,AGRICULTURA TECNICA,2005
11445,ARCHIVOS DE MEDICINA VETERINARIA,2005
11446,ARQ,2005
11447,ATENEA,2005
...,...,...
23787,SUSTAINABLE & BIODIVERSITY FRIENDLY VINEYARD L...,2013
23788,LONG-TERM EVOLUTION OF ALPINE GLACIATION AND T...,2013
23789,CRISPR-BASED TECHNOLOGIES FOR SYNTHETIC BIOLOG...,2013
23790,UCB-U. CHILE WORKING GROUP FOR COMPARATIVE EVO...,2013


Fijémonos sólo en los que son desde 2013 en adelante.

In [ ]:
df_valid.loc[(df_valid["CODIGO_PROYECTO"] == "SIN INFORMACION") &
             (df_valid["AGNO_FALLO"] >= 2013), ["NOMBRE_PROYECTO", "AGNO_FALLO"]]

,NOMBRE_PROYECTO,AGNO_FALLO
23760,K-12 AND HIGHER EDUCATION IN CHILE: SCHOOL SIZ...,2013
23761,BUILDING A NATURAL AND SOCIAL SCIENCE COLLABOR...,2013
23762,COLLABORATING TO QUANTIFY THE HEALTH BENEFITS ...,2013
23763,WHEN DOES RETIREMENT OPTIMIZE HEALTH? CAUSAL E...,2013
23764,PARENTAL INTERVENTIONS FOR HOME COMPUTERS IN C...,2013
23765,PUBLIC PROCUREMENT MECHANISMS: FROM PRACTICE T...,2013
23766,LEARNING FROM 27F: A COMPARATIVE ASSESSMENT OF...,2013
23767,DEGRADATION OF TITIN AT SINGLE MOLECULAR LEVEL,2013
23768,LANGUAGE FOR LEARNING: DESIGNING AND PILOT TES...,2013
23769,"PUBERTAL EVENTS, SUBSTANCE USE AND SEXUAL RISK...",2013


Son varios. Por ahora no nos preocuparemos, pero sí nos fijaremos que para analizar duplicados necesitamos tanto el código como la glosa del proyecto.

In [ ]:
df_class2 = df_class[~df_class.duplicated(subset=["código", "glosa"])]
df_class2

,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
41503,31180026,COSMOS IN THE KINDER: EMPOWERING SCIENTIFIC TH...,otro,otro
41510,31180032,DEVELOPING ASTRONOMY TOOLS FOR BLIND AND VISUA...,otro.,otro
41511,31180035,STAR FORMATION THROUGH TIME: FROM NEAR TO FAR,otro.,otro
41512,31180046,ANNUAL MEETING SOCHIAS 2020,otro.,otro


In [ ]:
df_class2["id"] = df_class2["código"] + " " + df_class2["glosa"]
df_class2

<ipython-input-86-39f1f2971a42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class2["id"] = df_class2["código"] + " " + df_class2["glosa"]


,código,glosa,clasificación,clasificación_normalizada,id
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro,ECOS190041 ARQUEOLOGIA Y AMBIENTE: INTERACCION...
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro,FP200006 IMPLEMENTACION DE FLUJO CONTINUO DE L...
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro,FOVI210059 LIQUENES SUBANTARTICOS UN NUEVO CEN...
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro,FOVI210061 CONSOLIDACION DE LA RED BINACIONAL ...
4,ED230064,PODCAST ANTARTICA,otro,otro,ED230064 PODCAST ANTARTICA
...,...,...,...,...,...
41503,31180026,COSMOS IN THE KINDER: EMPOWERING SCIENTIFIC TH...,otro,otro,31180026 COSMOS IN THE KINDER: EMPOWERING SCIE...
41510,31180032,DEVELOPING ASTRONOMY TOOLS FOR BLIND AND VISUA...,otro.,otro,31180032 DEVELOPING ASTRONOMY TOOLS FOR BLIND ...
41511,31180035,STAR FORMATION THROUGH TIME: FROM NEAR TO FAR,otro.,otro,31180035 STAR FORMATION THROUGH TIME: FROM NEA...
41512,31180046,ANNUAL MEETING SOCHIAS 2020,otro.,otro,31180046 ANNUAL MEETING SOCHIAS 2020


In [ ]:
df_valid["id"] = df_valid["CODIGO_PROYECTO"].astype(str) + " " + df_valid["NOMBRE_PROYECTO"]

<ipython-input-87-643a3338e1c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid["id"] = df_valid["CODIGO_PROYECTO"].astype(str) + " " + df_valid["NOMBRE_PROYECTO"]


In [ ]:
ids1 = set(df_valid["id"].unique())
ids2 = set(df_class2["id"].unique())

In [ ]:
ids1 - ids2

{'CET08 S/I'}

In [ ]:
ids2 - ids1

set()

In [ ]:
df_valid[df_valid.duplicated(subset=["CODIGO_PROYECTO", "NOMBRE_PROYECTO", "AGNO_FALLO"])]

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO,id
11949,SIN INFORMACION,11918,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA REGIONAL TIC-...,CONCURSO PROGRAMA REGIONAL TIC-AMSUD FRANCIA 2...,2005,2005,SIN INFORMACION,SIN INFORMACION,...,NaN,PERSONA NATURAL,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN,SIN INFORMACION SIN INFORMACION
13329,SIN INFORMACION,13298,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,REDES INTERNACIONALES - PROGRAMA REGIONAL TIC-...,CONCURSO PROGRAMA REGIONAL TIC-AMSUD FRANCIA 2...,2006,2006,SIN INFORMACION,SIN INFORMACION,...,NaN,PERSONA NATURAL,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,NaN,NaN,SIN INFORMACION SIN INFORMACION
16757,SIN INFORMACION,16726,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,CONICYT-CNRS/IRAM,CONCURSO PARA ASISTENCIA A LA ESCUELA DE RADIO...,2009,2009,SIN INFORMACION,CIENCIAS NATURALES,...,NaN,PERSONA NATURAL,PATRICIO GALLARDO MATAMALA,HOMBRE,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,RM,13. METROPOLITANA,NaN,NaN,SIN INFORMACION SIN INFORMACION
16758,SIN INFORMACION,16727,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,CONICYT-CNRS/IRAM,CONCURSO PARA ASISTENCIA A LA ESCUELA DE RADIO...,2009,2009,SIN INFORMACION,CIENCIAS NATURALES,...,NaN,PERSONA NATURAL,VIVIANA GUZMAN VELOSO,MUJER,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,NaN,NaN,SIN INFORMACION SIN INFORMACION
18359,SIN INFORMACION,18328,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,CONICYT-CNRS/IRAM,CONCURSO PARA ASISTENCIA A LA ESCUELA DE RADIO...,2010,2010,SIN INFORMACION,CIENCIAS NATURALES,...,NaN,PERSONA NATURAL,YANETT CONTRERAS,MUJER,UNIVERSIDAD DE CHILE,RM,13. METROPOLITANA,NaN,NaN,SIN INFORMACION SIN INFORMACION
19854,SIN INFORMACION,19823,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,CONICYT-CNRS,BECAS DE DOCTORADO EN ASTRONOMIA EN FRANCIA “C...,2011,2011,SIN INFORMACION,CIENCIAS NATURALES,...,NaN,PERSONA NATURAL,VALESKA VALDIVIA,MUJER,UNIVERSITY MARIE CURIE,SIN INFORMACION,SIN INFORMACION,NaN,NaN,SIN INFORMACION SIN INFORMACION
35095,ID17I20276,35072,INVESTIGACION APLICADA,FONDEF,IDEA DOS ETAPAS,IV CONCURSO IDEA EN DOS ETAPAS CONVOCATORIA 2017,2017,2020,DESARROLLO DE UN PROTOTIPO DE SISTEMA DE RECIR...,CIENCIAS NATURALES,...,NaN,PERSONA JURIDICA,CRISTIAN MEJIAS BARBOZA,HOMBRE,UNIVERSIDAD DE ANTOFAGASTA,NORTE,02. ANTOFAGASTA,200000,NaN,ID17I20276 DESARROLLO DE UN PROTOTIPO DE SISTE...


In [ ]:
len(df_valid)

41504

In [ ]:
len(df_class2)

41398

Parece que hay problemas con los códigos en la db original. Eso lo analizaremos en la próxima parte, pero por ahora, sabemos que sí procesamos cada llave código-nombre (excepto el proyecto CET08 que tiene título 'S/I'). Procedemos a guardar las clasificaciones.

In [ ]:
df_class2 = df_class2[["código", "glosa", "clasificación", "clasificación_normalizada"]]
df_class2

,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
41503,31180026,COSMOS IN THE KINDER: EMPOWERING SCIENTIFIC TH...,otro,otro
41510,31180032,DEVELOPING ASTRONOMY TOOLS FOR BLIND AND VISUA...,otro.,otro
41511,31180035,STAR FORMATION THROUGH TIME: FROM NEAR TO FAR,otro.,otro
41512,31180046,ANNUAL MEETING SOCHIAS 2020,otro.,otro


In [ ]:
df_class2.to_csv(os.path.join(save_dir, "clasificaciones_anid_enfermedades_especificas_limpia.csv"), index=False)